If you want to run this code, go to https://github.com/hmp-anthony/AIMA and look for AndOr.ipynb. In this notebook I am going to cherry-pick the `and_or_search` functionality from `search4e.ipynb` and expand upon it, adding more problems and advancing the code. 

In [1]:
import math

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    

failure = Node('failure', path_cost=math.inf)
loop = Node('loop', path_cost=math.inf)

In [2]:
class MultiGoalProblem(Problem):
    """A version of `Problem` with a colllection of `goals` instead of one `goal`."""
    
    def __init__(self, initial=None, goals=(), **kwds): 
        self.__dict__.update(initial=initial, goals=goals, **kwds)
        
    def is_goal(self, state): return state in self.goals
    
class ErraticVacuum(MultiGoalProblem):
    """In this 2-location vacuum problem, the suck action in a dirty square will either clean up that square,
    or clean up both squares. A suck action in a clean square will either do nothing, or
    will deposit dirt in that square. Forward and backward actions are deterministic."""
    
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = {'suck':{1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
             'forward': {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}},
             'backward': {1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}}}

class SlipperyErraticVacuum(MultiGoalProblem):
    """In this 2-location vacuum problem, the suck action in a dirty square will either clean up that square,
    or clean up both squares. A suck action in a clean square will either do nothing, or
    will deposit dirt in that square. Forward and backward actions are deterministic."""
    
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = {'suck':{1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
             'forward': {1:{1,2}, 2:{2}, 3:{3,4}, 4:{4}, 5:{5,6}, 6:{6}, 7:{7,8}, 8:{8}},
             'backward': {1:{1}, 2:{2,1}, 3:{3}, 4:{4,3}, 5:{5}, 6:{6,5}, 7:{7}, 8:{8,7}}}

In [3]:
def and_or_search(problem):
    "Find a plan for a problem that has nondterministic actions."
    return or_search(problem, problem.initial, [])
    
def or_search(problem, state, path):
    "Find a sequence of actions to reach goal from state, without repeating states on path."
    if problem.is_goal(state): return []
    if state in path: return failure
    for action in problem.actions(state):
        plan = and_search(problem, problem.results(state, action), [state] + path)
        if plan != failure:
            return [action] + plan
    return failure

def and_search(problem, states, path):
    "Plan for each of the possible states we might end up in."
    if len(states) == 1: 
        return or_search(problem, next(iter(states)), path)
    plan = {}
    for s in states:
        plan[s] = or_search(problem, s, path)
        if plan[s] == failure: return failure
    return [plan]

In [4]:
{s: and_or_search(ErraticVacuum(s, {7,8})) for s in range(1, 9)}

{1: ['suck', {5: ['forward', 'suck'], 7: []}],
 2: ['suck', {8: [], 4: ['backward', 'suck']}],
 3: ['suck'],
 4: ['backward', 'suck'],
 5: ['forward', 'suck'],
 6: ['suck'],
 7: [],
 8: []}

In [5]:
{s: and_or_search(SlipperyErraticVacuum(s, {7,8})) for s in range(1, 9)}

{1: <failure>,
 2: <failure>,
 3: ['suck'],
 4: <failure>,
 5: <failure>,
 6: ['suck'],
 7: [],
 8: []}

In [6]:
def test_acyclic(plan_string):
    return plan_string.find("<loop>") < 0

def and_or_search(problem):
    "Find a plan for a problem that has nondterministic actions."
    return or_search(problem, problem.initial, [])
    
def or_search(problem, state, path):
    "Find a sequence of actions to reach goal from state, without repeating states on path."
    if problem.is_goal(state): return []
    if state in path: return loop
    cycle_minus_plan = []
    for action in problem.actions(state):
        plan = and_search(problem, problem.results(state, action), [state] + path)
        if(plan != failure):
            if(type(plan) != list): plan = [plan]
            plan_string = str(plan)
            # return non-cyclic solution
            if(test_acyclic(plan_string)):
                return [action] + plan
            # save cyclic solution
            cycle_minus_plan.append([action] + plan)
    if cycle_minus_plan != None: return cycle_minus_plan
    return failure

def and_search(problem, states, path):
    "Plan for each of the possible states we might end up in."
    loopy = True
    if len(states) == 1: 
        return or_search(problem, next(iter(states)), path)
    plan = {}
    for s in states:
        plan[s] = or_search(problem, s, path)
        if plan[s] == failure: return failure
        if plan[s] != loop: loopy = False
    if not loopy: return [plan]
    return failure

In [31]:
{s: and_or_search(ErraticVacuum(s, {7,8})) for s in range(1, 9)}

{1: ['suck', {5: ['forward', 'suck'], 7: []}],
 2: ['suck', {8: [], 4: ['backward', 'suck']}],
 3: ['suck'],
 4: ['backward', 'suck'],
 5: ['forward', 'suck'],
 6: ['suck'],
 7: [],
 8: []}

In [37]:
D = {s: and_or_search(SlipperyErraticVacuum(s, {7,8})) for s in range(1, 9)}
print('1', D[1][0])
print('2', D[2][0])
print('3', D[3])
print('4', D[4][2])
print('5', D[5][1])
print('6', D[6])
print('7', D[7])
print('8', D[8])

1 ['suck', {5: [['forward', {5: <loop>, 6: ['suck']}], ['backward', <loop>]], 7: []}]
2 ['suck', {8: [], 4: [['forward', <loop>], ['backward', {3: ['suck'], 4: <loop>}]]}]
3 ['suck']
4 ['backward', {3: ['suck'], 4: <loop>}]
5 ['forward', {5: <loop>, 6: ['suck']}]
6 ['suck']
7 []
8 []


In [36]:
L = D[4][0]
for i in L:
    print(i)

suck
{2: [['suck', {8: [], 4: <loop>}], ['forward', <loop>], ['backward', {1: [['suck', {5: [['forward', {5: <loop>, 6: ['suck']}], ['backward', <loop>]], 7: []}], ['backward', <loop>]], 2: <loop>}]], 4: <loop>}
